In [1]:
import time
time.asctime( time.localtime(time.time()) )

'Tue Mar  7 13:35:54 2023'

In [3]:
import torch
from torch import nn
a = nn.Linear(128,128)
b = torch.empty((2,128,128))
a(b).shape
a = torch.as_tensor([1,2,3])
torch.prod(a)

tensor(6)

In [3]:
import numpy as np
np.sort(np.random.choice([1,3,5,7,9], 5 , replace = False))

array([1, 3, 5, 7, 9])

In [1]:
from utils.dataset import read_pic
import numpy as np
import cv2
p = read_pic("/home/supercgor/gitfile/data/bulkice/afm/T160_1", transform = True, rec_size= 0.02)
p.size()
p = p * 255
p = p.numpy()
p.astype(np.uint8)
p = np.reshape(p, (5,4, 128, 128))
out = np.concatenate(p, axis = 1)
out = np.transpose(out, (0,2,1))
out = np.concatenate(out, axis = 0)
cv2.imwrite("test2.png", out)

True

In [15]:
from config import get_config
a = get_config()
print(a)
a.merge_from_list(["data.dataset", "da"])
for i in a:
    print(i)

check_root: /home/supercgor/gitfile/data/model
data_root: /home/supercgor/gitfile/data
ovito: 
root: /home/supercgor/gitfile
save_dir: 
batch_size: 2
clip_grad: 10.0
device: [0]
epochs: 50
learning_rate: 0.0001
local_epoch: 999
max_save: 3
num_workers: 0
pin_memory: True
show: True
split: [0.0, 3.0]
channels: 32
checkpoint: 3A_with_more_data
input: 20
nms: True
output: 4
threshold: 0.5
dataset: bulkice
elem_name: ('O', 'H')
img_size: 128
img_use: 10
pred_filelist: pred.filelist
real_size: [25, 25, 3]
test_filelist: test.filelist
train_filelist: train.filelist
valid_filelist: valid.filelist
decay: [0.9, 0.7, 0.5, 0.3, 0.1, 0.05]
pos_weight: (5.0, 5.0)
reduction: mean
weight_confidence: 1.0
weight_offset_xy: 0.5
weight_offset_z: 0.5
